# Torch-Rechub Tutorial
- 安装torch—rechub环境
- 在新数据集上数据集训练一个DIN模型


In [1]:
# 安装torch-rechub
!rm -rf torch-rechub
!git clone https://github.com/datawhalechina/torch-rechub.git
%cd torch-rechub

!pip install -e .


fatal: destination path 'torch-rechub' already exists and is not an empty directory.
/Users/chester/Desktop/rechub-tutorial/torch-rechub
Obtaining file:///Users/chester/Desktop/rechub-tutorial/torch-rechub
  Attempting uninstall: torch-rechub
    Found existing installation: torch-rechub 0.0.1
    Uninstalling torch-rechub-0.0.1:
      Successfully uninstalled torch-rechub-0.0.1
  Running setup.py develop for torch-rechub


**除了torch和torch-rechub，我们依赖的库还有['numpy>=1.21.5', 'torch>=1.7.0', 'pandas>=1.0.5', 'tqdm>=4.64.0', 'scikit_learn>=0.23.2']**



In [2]:
# 检查torch的安装以及gpu的使用
import torch
print(torch.__version__, torch.cuda.is_available())

import torch_rechub
import pandas as pd
import numpy as np
import tqdm
import sklearn

1.10.0 False


## 在自定义数据集上训练DIN模型
训练新的模型只需要三个步骤：
- 支持新数据集
- 指定特征含义
- 训练新模型


### 支持新数据集
这里我们以Amazon-Electronics为例，原数据是json格式，我们提取所需要的信息预处理为一个仅包含user_id, item_id, cate_id, time四个特征列的CSV文件。

注意：examples文件夹中仅有100行数据方便我们轻量化学习，如果需要Amazon数据集全量数据用于测试模型性能有两种方法：
1. 我们提供了处理完成的全量数据在高速网盘链接：https://cowtransfer.com/s/e911569fbb1043 ，只需要下载全量数据后替换下一行的file_path即可；
2. 前往Amazon数据集官网：http://jmcauley.ucsd.edu/data/amazon/index_2014.html ，进入后选择elextronics下载，我们同样提供了数据集处理脚本在examples/ranking/data/amazon-electronics/preprocess_amazon_electronics.py文件中。

In [3]:
# 查看文件
file_path = 'examples/ranking/data/amazon-electronics/amazon_electronics_sample.csv'
data = pd.read_csv(file_path)
data

,user_id,item_id,time,cate_id
0,41064,13179,1396656000,584
1,89202,13179,1380499200,584
2,95407,13179,1364688000,584
3,101617,13179,1389657600,584
4,174964,13179,1363478400,584
...,...,...,...,...
94,2974,29247,1365724800,339
95,3070,29247,1294790400,339
96,3139,29247,1388448000,339
97,3192,29247,1359590400,339


In [4]:
from torch_rechub.utils.data import create_seq_features
# 构建用户的历史行为序列特征，内置函数create_seq_features只需要指定数据，和需要生成序列的特征，drop_short是选择舍弃行为序列较短的用户
train, val, test = create_seq_features(data, seq_feature_col=['item_id', 'cate_id'], drop_short=0)
# 查看当前构建的序列
train

,user_id,history_item,history_cate,target_item,target_cate,label
0,1,"[2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3,0
1,1,"[2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,1,1


### 指定特征含义
在DIN模型中，我们讲使用两种类别的特征，分别是类别特征和序列特征。对于类别特征，我们希望模型将其输入Embedding层，而对于序列特征，我们不仅希望模型将其输入Embedding层，还需要计算target-attention分数，所以需要指定DataFrame中每一列的含义，让模型能够正确处理。


在这个案例中，我们使用user_id,item_id和item_cate这三个类别特征，使用用户的item_id和cate的历史序列作为序列特征。在torch-rechub我们只需要调用DenseFeature, SparseFeature, SequenceFeature这三个类，就能自动正确处理每一类特征。

In [5]:
from torch_rechub.basic.features import DenseFeature, SparseFeature, SequenceFeature

n_users, n_items, n_cates = data["user_id"].max(), data["item_id"].max(), data["cate_id"].max()
# 这里指定每一列特征的处理方式，对于sparsefeature，需要输入embedding层，所以需要指定特征空间大小和输出的维度
features = [SparseFeature("target_item", vocab_size=n_items + 2, embed_dim=8),
            SparseFeature("target_cate", vocab_size=n_cates + 2, embed_dim=8),
            SparseFeature("user_id", vocab_size=n_users + 2, embed_dim=8)]
target_features = features
# 对于序列特征，除了需要和类别特征一样处理意外，item序列和候选item应该属于同一个空间，我们希望模型共享它们的embedding，所以可以通过shared_with参数指定
history_features = [
    SequenceFeature("history_item", vocab_size=n_items + 2, embed_dim=8, pooling="concat", shared_with="target_item"),
    SequenceFeature("history_cate", vocab_size=n_cates + 2, embed_dim=8, pooling="concat", shared_with="target_cate")
]

In [6]:
from torch_rechub.utils.data import df_to_dict, DataGenerator
# 指定label，生成模型的输入，这一步是转换为字典结构
train = df_to_dict(train)
val = df_to_dict(val)
test = df_to_dict(test)

train_y, val_y, test_y = train["label"], val["label"], test["label"]

del train["label"]
del val["label"]
del test["label"]
train_x, val_x, test_x = train, val, test

# 最后查看一次输入模型的数据格式
train_x

# 构建dataloader，指定模型读取数据的方式，和区分验证集测试集、指定batch大小
dg = DataGenerator(train_x, train_y)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=val_x, y_val=val_y, x_test=test_x, y_test=test_y, batch_size=16)

### 训练新模型
我们封装了召回、排序、多任务等众多工业界主流的模型，基本能够做到几个参数定义一个模型。

在本案例中，我用训练一个深度兴趣网络DIN模型，我们只需要指定DIN的少数模型结构参数，和学习率等参数，就可以完成训练。

In [7]:
from torch_rechub.models.ranking import DIN
from torch_rechub.trainers import CTRTrainer

# 定义模型，模型的参数需要我们之前的feature类，用于构建模型的输入层，mlp指定模型后续DNN的结构，attention_mlp指定attention层的结构
model = DIN(features=features, history_features=history_features, target_features=target_features, mlp_params={"dims": [256, 128]}, attention_mlp_params={"dims": [256, 128]})

# 模型训练，需要学习率、设备等一般的参数，此外我们还支持earlystoping策略，及时发现过拟合
ctr_trainer = CTRTrainer(model, optimizer_params={"lr": 1e-3, "weight_decay": 1e-3}, n_epoch=3, earlystop_patience=4, device='cpu', model_path='./')
ctr_trainer.fit(train_dataloader, val_dataloader)

# 查看在测试集上的性能
auc = ctr_trainer.evaluate(ctr_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epoch: 0


validation: 100%|██████████| 1/1 [00:12<00:00, 12.00s/it]


epoch: 0 validation: auc: 0.0
epoch: 1


validation: 100%|██████████| 1/1 [00:09<00:00,  9.37s/it]


epoch: 1 validation: auc: 0.0
epoch: 2


validation: 100%|██████████| 1/1 [00:09<00:00,  9.35s/it]


epoch: 2 validation: auc: 0.0


validation: 100%|██████████| 1/1 [00:10<00:00, 10.41s/it]

test auc: 1.0
